### Imports

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import tqdm

# custom packages
from ratsimulator import trajectory_generator
from ratsimulator.Environment import Rectangle

import datetime
import sys
import os
# avoid adding multiple relave paths to sys.path
sys.path.append("../src") if "../src" not in sys.path else None 

from PlaceCells import PlaceCells
from Models import SorscherRNN
from methods import *
from datahandling import *
import spatial_maps as sm # CINPLA spatial maps

### Set parameters

In [3]:
"""
# Sorscher params
options.save_dir = '/mnt/fs2/bsorsch/grid_cells/models/'
options.n_steps = 100000      # number of training steps
options.batch_size = 200      # number of trajectories per batch
options.sequence_length = 20  # number of steps in trajectory
options.learning_rate = 1e-4  # gradient descent learning rate
options.Np = 512              # number of place cells
options.Ng = 4096             # number of grid cells
options.place_cell_rf = 0.12  # width of place cell center tuning curve (m)
options.surround_scale = 2    # if DoG, ratio of sigma2^2 to sigma1^2
options.RNN_type = 'RNN'      # RNN or LSTM
options.activation = 'relu'   # recurrent nonlinearity
options.weight_decay = 1e-4   # strength of weight decay on recurrent weights
options.DoG = True            # use difference of gaussians tuning curves
options.periodic = False      # trajectories with periodic boundary conditions
options.box_width = 2.2       # width of training environment
options.box_height = 2.2      # height of training environment
"""
_ = 2 # prevent jupyter from printing docstring (messy)

In [4]:
params = {}
# Environment params
params['boxsize'] = (2.2, 2.2)
params['origo'] = (0,0)
params['soft_boundary'] = 0.03 # Sorscher uses 0.03, I used to have 0.2
# Remapping / meta-learning (multiple environments/contexts)
params['num_environments'] = 1
params['sampler'] = 'MESampler'
# Place Cells params
params['npcs'] = 512 # as used in Sorscher model
params['pc_width'] = 0.12
params['DoG'] = True
params['pcc_seeds'] = list(range(params['num_environments'])) # place-cell center seeds
# Training data (Agent) params
params['batch_size'] = 200
params['seq_len'] = 20
params['angle0'] = None # random
params['p0'] = None     # random
# Agent/random walk parameters
params['dt'] = 0.02
params['turn_angle'] = 5.76 * 2
params['b'] = 0.13 * 2 * np.pi
params['mu'] = 0
params['boundary_mode'] = 'sorschers' # 'sorschers' or 'zenos'
# Model params
params['Ng'] = 4096
params['Np'] = params['npcs'] # defined for Brain already
params['weight_decay'] = 1e-5 # Sorscher uses 1e-4 in his publised code - but says he uses 1e-5 in the paper (only for LSTM model?)
params['lr'] = 1e-4# 1e-3 is default for Adam()
params['nsteps'] = 100 # number of mini batches in an epoch
params['nepochs'] = 1000 # number of epochs
# metadata
params['tag'] = 'default_with_xavier'
params['date'] = datetime.datetime.now()

num_workers = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"{device=}")

device=device(type='cuda')


### Initialise objects

In [19]:
# Initialise environments
environments = [Rectangle(boxsize=params['boxsize'], soft_boundary=params['soft_boundary'])] * params['num_environments']
params['environment_names'] = [type(environment).__name__ for environment in environments]

# Initialise agents
agents = [Agent(environment = environment, **params) for environment in environments]

# Initialise brain representations
place_cell_ensembles = [PlaceCells(environment = environment, seed = seed, **params) for environment, seed in zip(environments, params['pcc_seeds'])]

# Initialise data loading
num_samples = params['nsteps'] * params['nepochs']
dataset = Dataset(agents = agents, place_cell_ensembles = place_cell_ensembles, num_samples = num_samples, **params)
datasampler = eval(params['sampler'])(num_environments = len(environments), num_samples = num_samples)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=params['batch_size'], sampler = datasampler, num_workers=num_workers)

# Initialise model
model = SorscherRNN(place_cell_ensembles, Ng=params['Ng'], Np=params['Np']) 
model.to(device)
print(f"{model = }")

# Initialise optimizer (use custom weight decay, rather than torch optim decay)
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], betas=(0.9, 0.999), \
                             eps=1e-08, weight_decay=0.0, amsgrad=False)

Singular matrix
Singular matrix
model = SorscherRNN(
  (init_position_encoder): Linear(in_features=512, out_features=4096, bias=False)
  (RNN): RNN(2, 4096, bias=False, batch_first=True)
  (decoder): Linear(in_features=4096, out_features=512, bias=False)
)


### Data pathing. To save/load model weights and results

In [21]:
# create model model tag
model_tag = type(model).__name__ + "_" + params["tag"]
print(f"{model_tag = }")

# create relevant paths
from pathlib import Path
data_path = Path().home() / "data"
project_path = data_path / "emergent-grid-cells"
model_path = project_path / model_tag
checkpoint_path = model_path / "checkpoints"
pickle_path = model_path / "pickle_objects"
dynamics_path = model_path / "dynamics"
print(f"{checkpoint_path = }")
print(f"{pickle_path = }")
print(f"{dynamics_path = }")

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
    os.makedirs(pickle_path)
    os.makedirs(dynamics_path)

model_tag = 'SorscherRNN_default_with_xavier'
checkpoint_path = PosixPath('/home/users/vemundss/data/emergent-grid-cells/SorscherRNN_default_with_xavier/checkpoints')
pickle_path = PosixPath('/home/users/vemundss/data/emergent-grid-cells/SorscherRNN_default_with_xavier/pickle_objects')
dynamics_path = PosixPath('/home/users/vemundss/data/emergent-grid-cells/SorscherRNN_default_with_xavier/dynamics')


### Train/Load Model

In [26]:
loss_history = []
training_metrics = {}

model_checkpoints = [f for f in os.listdir(checkpoint_path) if os.path.isfile(os.path.join(checkpoint_path, f))]
if model_checkpoints:
    # get latest (wrt. #epochs trained) model weights
    model_checkpoints.sort()
    checkpoint_epoch = model_checkpoints[-1]
    # overwrite and choose manually which weights to load
    #checkpoint_epoch = "0001"
    
    # load model
    print(f"Loading model at epoch = {checkpoint_epoch}")
    checkpoint = torch.load(checkpoint_path / checkpoint_epoch)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss_history = checkpoint['loss_history']
    training_metrics = checkpoint['training_metrics']
    print("Loaded weights")
    
# whether to train
if train:=True:
    loss_history = model.train(trainloader = dataloader, optimizer = optimizer, weight_decay=params['weight_decay'], \
                               nepochs=params['nepochs'], checkpoint_path = checkpoint_path, params = params, \
                               loss_history = loss_history, training_metrics = training_metrics)

  0%|          | 0/1000 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

  0%|          | 0/1000 [00:27<?, ?it/s]

582
583


KeyboardInterrupt: 

## **Analyse Model**
### Training history

In [ ]:
fig, ax = plt.subplots(ncols=5,figsize=(15,4))
ax[0].plot(loss_history)
ax[0].set_title('Total-loss history')

for i,(key,value) in enumerate(checkpoint['training_metrics'].items()):
    if key == 'KL':
        ax[i+1].axhline(0,ls=":")
    ax[i+1].plot(value)
    ax[i+1].set_title(key)
plt.tight_layout()

# maximum labelled distribution entropy (uniform labelled distribution)
n = 512
px = np.ones(n) / n # uniform
entropy = lambda x: -np.sum(x * np.log(x))
print(f"Maximum Entropy possible: {entropy(px)}")

### Create/Load ratemaps

In [ ]:
ratemaps_path = pickle_path / "ratemaps"
latest_ratemaps_path = ratemaps_path / (model_checkpoints[-1] + ".pkl")
if os.path.exists(latest_ratemaps_path):
    # load ratemaps generated by model that has trained the LONGEST
    with open(latest_ratemaps_path, "rb") as f:
        ratemaps = pickle.load(f)
    print("Loaded ratemaps")
else:
    # create ratemaps for ALL checkpoints of model
    idxs = slice(0, params['Ng'], 1)
    res = np.array([32, 32])
    print("Calculating ratemaps")
    for checkpoint_epoch in tqdm.tqdm(model_checkpoints):
        epoch_ratemaps_path = ratemaps_path / (checkpoint_epoch + ".pkl")
        if os.path.exists(epoch_ratemaps_path):
            # ratemaps for this model checkpoint already calculated and saved
            continue
            
        checkpoint = torch.load(checkpoint_path / checkpoint_epoch)
        model.load_state_dict(checkpoint['model_state_dict'])
        ratemaps = compute_ratemaps(model=model.g, dataset=dataset, num_trajectories=1250, res=res, idxs=idxs)[0]
        
        with open(epoch_ratemaps_path, "wb") as f:
            pickle.dump(ratemaps,f)
        # print("Saved:", checkpoint_epoch + "_ratemaps.pkl")

### Grid score ratemaps (only last model checkpoint)

In [ ]:
def score_ratemaps(ratemaps):
    # if grid scores have already been calculate and saved to disk, load these (FAST!)
    score_map_path = pickle_path / (model_checkpoints[-1] + "_grid_scores.pkl")
    if os.path.exists(score_map_path):
        print(os.path.split(score_map_path)[-1], "already exists. Loading score map.")
        with open(score_map_path, "rb") as f:
            score_map = pickle.load(f)
        return score_map
    
    # BANINO (and Sorscher) grid scoring method
    from scores import GridScorer
    res = np.array([32,32])
    starts = [0.2] * 10
    ends = np.linspace(0.4, 1.0, num=10)
    box_width, box_height = params['boxsize']
    coord_range=((params['origo'][0], box_width), (params['origo'][0], box_height))
    coords_range=((-box_width/2, box_width/2), (-box_height/2, box_height/2))
    mask_parameters = zip(starts, ends.tolist())
    scorer = GridScorer(nbins=res[0], coords_range=coords_range, mask_parameters=mask_parameters)

    # score ratemaps (Could use NUMBA for faster computations?)
    score_map = np.zeros(params['Ng']) # ratemap score lookup table
    print("Calculating grid score for the ratemaps")
    for i in tqdm.trange(params['Ng']):
        # interpolated_ratemap = interpolate_missing_pixels(ratemaps[i], np.isnan(ratemaps[i]))
        # score_map[i] = sm.gridness(interpolated_ratemap)
        score_60, score_90, max_60_mask, max_90_mask, sac = scorer.get_scores(ratemaps[i])
        score_map[i] = score_60
        
    # make dirs and pickle ratemaps
    if not os.path.exists(pickle_path):
        os.makedirs(pickle_path)
    with open(score_map_path, "wb") as f:
        pickle.dump(score_map,f)
    return score_map

score_map = score_ratemaps(ratemaps)

# sort scores and ratemaps
sort_idxs = np.argsort(score_map)
sorted_scores = score_map[sort_idxs][::-1]
sorted_ratemaps = ratemaps[sort_idxs][::-1]

### Create grid cell ratemap dynamics (grid cell emergence video)

In [ ]:
if not os.path.exists(dynamics_path):
    # create videos
    os.makedirs(dynamics_path)
    ratemap_dynamics = []
    print("Loading ratemaps")
    for checkpoint_epoch in tqdm.tqdm(model_checkpoints):
        with open(ratemaps_path / (checkpoint_epoch + ".pkl"), "rb") as f:
            # load and sort ratemaps wrt. grid score
            ratemap_dynamics.append(pickle.load(f)[sort_idxs][::-1])
    ratemap_dynamics = np.stack(ratemap_dynamics, axis=0)

    from dynamics import RatemapDynamics
    num_gc_in_video = 64
    num_videos = params['Ng'] // num_gc_in_video
    print("Creating videos")
    for gcs_i in tqdm.trange(0, num_videos):
        idxs = slice(gcs_i * num_gc_in_video, (gcs_i + 1) * num_gc_in_video)
        ratemap_video = RatemapDynamics(ratemap_dynamics[:,idxs], sorted_scores[[idxs.start, idxs.stop - 1]])
        ratemap_video.animation.save(dynamics_path / f'{idxs.start}-{idxs.stop - 1}.mp4')

In [ ]:
start_idx = 0
num_ratemaps = 256
fig, ax = multiimshow(sorted_ratemaps[start_idx:start_idx+num_ratemaps])
fig.suptitle(f"gc_score = {sorted_scores[start_idx]} -- {sorted_scores[start_idx+num_ratemaps]}", fontsize=16)
fig, ax = multiimshow(sorted_ratemaps[start_idx+num_ratemaps:start_idx+2*num_ratemaps])
fig.suptitle(f"gc_score = {sorted_scores[start_idx+num_ratemaps]} -- {sorted_scores[start_idx+2*num_ratemaps]}", fontsize=16)

In [ ]:
start_idx = 256*1
num_ratemaps = 256
fig, ax = multiimshow(ratemaps[start_idx:start_idx+num_ratemaps])

### Predicted place Cells (with(out) SOFTMAX)

In [ ]:
"""
forward_with_softmax = lambda x: torch.exp(model(x, log_softmax=True))
ratemaps = compute_ratemaps(model=model, dataset=dataset, num_trajectories=1250, res=res, idxs=idxs)[0]
fig, ax = multiimshow(ratemaps)
"""

In [ ]:
model.prune_mask = list(range(int(4096/2),4096)) # set prune mask
model.prune_mask = [] # reset prune mask
model.prune_mask

In [ ]:
idxs=slice(0, 16**2, 1)
res=np.array([32, 32])
ratemaps = compute_ratemaps(model=model.g, dataset=dataset, num_trajectories=1250, res=res, idxs=idxs)[0]
fig, ax = multiimshow(ratemaps)

### Decoding labels and predictions to cartesian

In [ ]:
[[vel, init_pos], labels] = dataset[0]
true_cartesian_pos = dataset.agent.positions
true_decoded_pos = place_cells.to_euclid(torch.cat([init_pos[None], labels]))
pc_preds = model([vel, init_pos]).detach().cpu()[0]
predicted_decoded_pos = place_cells.to_euclid(torch.cat([init_pos[None], pc_preds]))

In [ ]:
fig, ax = plt.subplots()
ax.plot(*true_cartesian_pos.T, label='true_cartesian_pos')
ax.plot(*true_decoded_pos.T, label='true_decoded_pos')
ax.plot(*predicted_decoded_pos.T, label='predicted_decoded_pos', ls=':')
environment.plot_board(ax)
ax.legend()

### Plot all place cell centers and some with tuning curves

In [ ]:
fig, ax = plt.subplots()
x,y = place_cells.pcs.T

ax.plot(x, y, "+")
# add standard deviation circles to locations
for i in range(5):
    ax.plot(x[i], y[i], "r+")
    a_circle = plt.Circle((x[i], y[i]), params['pc_width'], fill=False, color=(1, 0, 0, 0.5))
    ax.add_artist(a_circle)

plt.title("Spatial plot of place cell locations")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

### Calculate grid scores using different implementations of the metric

In [ ]:
# Custom grid score
print("CUSTOM:", grid_score(ratemaps[1])) 

# CINPLA grid score
import spatial_maps as sm
print("CINPLA:", sm.gridness(ratemaps[1])) 

# BANINO (and Sorscher) grid scoring
from scores import GridScorer
"""
One difference from custom and CINPLA grid scores: 
1. Uses average difference between phase60 and phase30 correlations
"""
starts = [0.2] * 10
ends = np.linspace(0.4, 1.0, num=10)
coord_range=((0, environment.boxsize[0]), (0, environment.boxsize[1]))
box_width, box_height = 2.2, 2.2
coords_range=((-box_width/2, box_width/2), (-box_height/2, box_height/2))
mask_parameters = zip(starts, ends.tolist())
scorer = GridScorer(nbins=res[0], coords_range=coords_range, mask_parameters=mask_parameters)

#score_60, score_90, max_60_mask, max_90_mask, sac, max_60_ind = zip(
#      *[scorer.get_scores(rm.reshape(res, res)) for rm in tqdm(rate_map_lores)])
score_60, score_90, max_60_mask, max_90_mask, sac = scorer.get_scores(ratemaps[1])
print("BANINO/SORSCHER:", score_60)

In [ ]:
# choose grid scoring function to use, e.g: grid_score, sm.gridness or scorer.get_scores
# for scorer.get_scores use: < (lambda rm: scorer.get_scores(rm)[0])(rate_map) >
grid_scoring_fn = lambda rate_map: sm.gridness(rate_map)

#map(grid_scoring_fn, *ratemaps)
grid_scoring_fn(ratemaps[1])

### Small analysis / checks / tests etc

In [ ]:
# Wr = model.recurrence.weight.detach().cpu().numpy()
Wr = model.RNN.weight_hh_l0.detach().cpu().numpy()
stats = lambda W : print(f"{np.min(W)=}, {np.max(W)=}, {np.min(abs(W))=}, {np.mean(W)=}, {np.std(W)=}, {np.sum(W**2)=}")
stats(Wr)
plt.imshow(Wr[:25,:25])
plt.colorbar()

In [ ]:
Wp = model.init_position_encoder.weight.detach().cpu().numpy()
stats = lambda W : print(f"{np.min(W)=}, {np.max(W)=}, {np.min(abs(W))=}, {np.mean(W)=}, {np.std(W)=}, {np.sum(W**2)=}")
stats(Wp)
plt.imshow(Wp)
plt.colorbar()